In [1]:
import pandas as pd
from metasynth import MetaDataset, MetaVar
from metasynth.distribution.util import _get_all_distributions
import wget
from pathlib import Path
import json

In [2]:
dtypes = {
    "Survived": "category",
    "Pclass": "category",
    "Name": "string",
    "Sex": "category",
    "SibSp": "category",
    "Parch": "category",
    "Ticket": "string",
    "Cabin": "string",
    "Embarked": "category"
}

In [3]:
titanic_fp = Path("titanic.csv")
if not titanic_fp.is_file():
    wget.download("https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv")

In [4]:
df = pd.read_csv(titanic_fp, dtype=dtypes)

In [5]:
dataset = MetaDataset.from_dataframe(df)

In [6]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,<NA>,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,<NA>,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,<NA>,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,<NA>,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,<NA>,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [7]:
var = MetaVar.detect(df["Cabin"])

In [8]:
distributions = _get_all_distributions("metasynth.distribution")

In [9]:
var.fit(distributions)

In [10]:
dataset.synthesize(1000)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,2,"SJz9K T5y(Zph""o;e:QC}4aq8eCOoJt~Ok",female,30.675038,0,0,2,45.729195,A2238,S
1,2,1,1,"I;xagX z rzwh]@L,^bfcf-m7txa-:^X]q/.5'c Bd4I! ...",male,24.254221,0,1,061281,60.360047,<NA>,C
2,3,0,3,oBfJ{:V!UK#v:1kKWZ98En7g(~a{Y1eS@VG9 0Q,male,38.858653,0,0,878,27.551469,<NA>,C
3,4,0,3,"L?/@,[YJBS/J0NI ""ci]{@&>R1 Ajyj@-I!JvTrU5s[8 O...",male,NaN,0,0,90,24.874455,<NA>,S
4,5,1,1,9)\QX3y ic-8yE:pk_Uc-KvK+cjMQ~ux,male,47.717435,0,0,3,48.487277,P834,S
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,1,3,"dL,QW[f7{8e QIK+*O>vtp,nA; M7TO>~SF5d%m@|Sp4p@...",female,22.236442,1,0,396,66.121305,<NA>,C
996,997,0,3,^<g d-UHN(LW*#B9fES' <. h@>nC7vD -BOT,male,NaN,0,0,826781,29.906021,<NA>,S
997,998,0,1,IN(2f\H]gJRFx__'Z gPiNb@ 1]gtSR},female,3.992533,0,0,;X>{AB j)3988094,49.164514,D0,S
998,999,1,3,"GZ -v{ L9ia4YlUjCwv)j_L6;$t.&r5l;-7K{&o4)>c,#v...",male,20.214602,0,0,0989351,58.786638,<NA>,Q


In [11]:
dataset.to_json("test.json", validate=True)

ValidationError: 'discrete' is not one of ['IntVar', 'FloatVar', 'StringVar', 'CategoricalVar']

Failed validating 'enum' in schema['properties']['vars']['items']['properties']['type']:
    {'enum': ['IntVar', 'FloatVar', 'StringVar', 'CategoricalVar']}

On instance['vars'][0]['type']:
    'discrete'

In [ ]:
print(MetaDataset.from_json("test.json"))

In [ ]:
dataset.to_dict()

In [ ]:
print(MetaDataset.from_xml("test.xml"))

In [ ]:
dataset.to_dict()